In [ ]:
def recommender(product):
    similarity_scores = []
    binary_list = []

    idx = df[df['skincare_name'] == product].index.item()

    for i in matrix_ingredients.iloc[idx][1:]:
        binary_list.append(i)   

    p1 = np.array(binary_list).reshape(1, -1)
    p1 = [val for sublist in p1 for val in sublist]
    
    prod_type = df['skincare_type'][df['skincare_name'] == product].iat[0]
    brand_search = df['brand'][df['skincare_name'] == product].iat[0]

    data_by_type = df[df['skincare_type'] == prod_type]
    
    for j in range(data_by_type.index[0], data_by_type.index[0] + len(data_by_type)):
        binary_list2 = []
        for k in matrix_ingredients.iloc[j][1:]:
            binary_list2.append(k)
        p2 = np.array(binary_list2).reshape(1, -1)
        p2 = [val for sublist in p2 for val in sublist]

        dot_product = np.dot(p1, p2)
        norm_1 = np.linalg.norm(p1)
        norm_2 = np.linalg.norm(p2)
        cos_sim = dot_product / (norm_1 * norm_2)
        similarity_scores.append(cos_sim)

    data_by_type = pd.DataFrame(data_by_type)
    data_by_type['cos_sim'] = similarity_scores

    data_by_type = data_by_type.sort_values('cos_sim', ascending=False)
    
    data_by_type = data_by_type[data_by_type.skincare_name != product] 
    
    x = 0
    brands = []
    output = []
    ingredients_list = []

    for m in range(len(data_by_type)):
        brand = data_by_type['brand'].iloc[x]
        if len(brands) == 0:
            if brand != brand_search:
                brands.append(brand)
                output.append(data_by_type.iloc[x])
                ingredients_list.append(data_by_type['ingredients'].iloc[x])
        elif brands.count(brand) < 2:
            if brand != brand_search:
                brands.append(brand)
                output.append(data_by_type.iloc[x])
                ingredients_list.append(data_by_type['ingredients'].iloc[x])
        x += 1

    df5 = pd.DataFrame(output)['ingredients'].head(5)

# Split the ingredients into separate values and flatten the list
    ingredients_list = df5.str.split(', ').sum()

# Count the occurrences of each ingredient
    ingredient_counts = pd.Series(ingredients_list).value_counts()

# Print the most frequently appearing ingredients with their counts
    for ingredient, count in ingredient_counts.head(5).items():
        print(f"{ingredient}: {count}")